In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import citypy module to determine city name from coordinates
from citipy import citipy
# import datetime
from datetime import datetime
import requests
from config import weather_api_key
import time
# Import linear regression dependency
from scipy.stats import linregress
import os

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list to hold city names
cities = []

# Identify the cities for each longitude and latitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If city is unique we will add it to the cities list
    if city not in cities:
        cities.append(city)
# Print city count to confirm
print(len(cities))


608


In [4]:
city_data = []
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging
print('Beginning data retrieval    ')
print('----------------------------')

# Create counters
record_count = 1
set_count = 1
counter = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
     # Run an API request for each of the cities.
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        city_precip = city_weather['weather'][0]['main']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        try:
            rain_3h = city_weather['rain']['3h']
            rain_3h_inches = rain_3h / 25.4
        except:
            rain_3h_inches = 0
            pass
        try:
            snow_3h = city_weather['snow']['3h']
            snow_3h_inches = snow_3h / 25.4
        except:
            snow_3h_inches = 0
            pass
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather Description": city_weather_description,
                          "City Precip": city_precip,
                          "Rain in Inches": rain_3h_inches,
                          "Snow in Inches": snow_3h_inches})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
    
    #city_data[counter].update({"Rain in Inches":rain_3h_inches})
    #counter +=1          
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

Beginning data retrieval    
----------------------------
Processing Record 1 of Set 1 | souillac
Processing Record 2 of Set 1 | jining
Processing Record 3 of Set 1 | khatanga
Processing Record 4 of Set 1 | luceville
City not found. Skipping...
Processing Record 5 of Set 1 | belyy yar
Processing Record 6 of Set 1 | vanimo
Processing Record 7 of Set 1 | avarua
Processing Record 8 of Set 1 | iqaluit
Processing Record 9 of Set 1 | hobyo
Processing Record 10 of Set 1 | puerto ayora
Processing Record 11 of Set 1 | makat
Processing Record 12 of Set 1 | albany
Processing Record 13 of Set 1 | castro
Processing Record 14 of Set 1 | nhulunbuy
Processing Record 15 of Set 1 | lloret de mar
Processing Record 16 of Set 1 | cape town
Processing Record 17 of Set 1 | paamiut
Processing Record 18 of Set 1 | dikson
Processing Record 19 of Set 1 | diplo
Processing Record 20 of Set 1 | kavaratti
Processing Record 21 of Set 1 | rikitea
Processing Record 22 of Set 1 | saint-philippe
Processing Record 23 of S

In [5]:
# Converst json to DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description,City Precip,Rain in Inches,Snow in Inches
0,Souillac,-20.52,57.52,77.00,94,40,2.24,MU,2020-04-02 21:51:13,scattered clouds,Clouds,0.000000,0.0
1,Jining,35.41,116.58,39.15,80,45,1.99,CN,2020-04-02 21:51:14,scattered clouds,Clouds,0.000000,0.0
2,Khatanga,71.97,102.50,15.87,91,100,18.07,RU,2020-04-02 21:51:14,overcast clouds,Clouds,0.000000,0.0
3,Belyy Yar,53.60,91.39,28.40,74,17,2.24,RU,2020-04-02 21:51:14,smoke,Smoke,0.000000,0.0
4,Vanimo,-2.67,141.30,79.32,84,99,8.77,PG,2020-04-02 21:51:15,light rain,Rain,0.062992,0.0


In [7]:
city_data_df.columns=['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed','Country','Date','Current Weather','Type of Weather', 'Rain Inches(last 3h)', 'Snow Inches(last 3h)']

In [8]:
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,Souillac,-20.52,57.52,77.00,94,40,2.24,MU,2020-04-02 21:51:13,scattered clouds,Clouds,0.000000,0.0
1,Jining,35.41,116.58,39.15,80,45,1.99,CN,2020-04-02 21:51:14,scattered clouds,Clouds,0.000000,0.0
2,Khatanga,71.97,102.50,15.87,91,100,18.07,RU,2020-04-02 21:51:14,overcast clouds,Clouds,0.000000,0.0
3,Belyy Yar,53.60,91.39,28.40,74,17,2.24,RU,2020-04-02 21:51:14,smoke,Smoke,0.000000,0.0
4,Vanimo,-2.67,141.30,79.32,84,99,8.77,PG,2020-04-02 21:51:15,light rain,Rain,0.062992,0.0


## How many cities have recorded rain or snow?

In [18]:
rain_snow = city_data_df.loc[(city_data_df['Type of Weather'] == 'Rain') | (city_data_df['Type of Weather'] == 'Snow')]
rain_snow.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
4,Vanimo,PG,2020-04-02 21:51:15,-2.67,141.30,79.32,84,99,8.77,light rain,Rain,0.062992,0.000000
9,Makat,KZ,2020-04-02 21:51:16,47.65,53.35,44.96,53,99,13.91,light rain,Rain,0.004724,0.000000
10,Albany,US,2020-04-02 21:50:04,42.60,-73.97,53.60,40,90,13.87,shower rain,Rain,0.000000,0.000000
16,Dikson,RU,2020-04-02 21:51:18,73.51,80.55,21.42,96,100,27.29,light snow,Snow,0.000000,0.012992
19,Rikitea,PF,2020-04-02 21:51:19,-23.12,-134.97,76.96,74,60,6.71,light rain,Rain,0.046850,0.000000


In [19]:
rain_snow.tail()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
523,Dossor,KZ,2020-04-02 21:53:56,47.53,52.98,44.01,58,100,15.86,light rain,Rain,0.012205,0.0
537,Formosa Do Rio Preto,BR,2020-04-02 21:54:01,-11.05,-45.19,74.95,78,100,3.27,light rain,Rain,0.049606,0.0
542,Kenai,US,2020-04-02 21:52:52,60.55,-151.26,35.60,86,90,8.05,light rain,Rain,0.000000,0.0
550,Maragogi,BR,2020-04-02 21:52:15,-9.01,-35.22,80.29,81,100,8.86,light rain,Rain,0.048819,0.0
551,Acarau,BR,2020-04-02 21:54:04,-2.89,-40.12,80.08,86,100,7.99,light rain,Rain,0.007087,0.0


In [17]:
rain_snow.count()

City                    133
Country                 133
Date                    133
Lat                     133
Lng                     133
Max Temp                133
Humidity                133
Cloudiness              133
Wind Speed              133
Current Weather         133
Type of Weather         133
Rain Inches(last 3h)    133
Snow Inches(last 3h)    133
dtype: int64

In [9]:
city_data_df = city_data_df[['City','Country','Date','Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness','Wind Speed', 'Current Weather','Type of Weather', 'Rain Inches(last 3h)','Snow Inches(last 3h)']]

In [10]:
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,Souillac,MU,2020-04-02 21:51:13,-20.52,57.52,77.00,94,40,2.24,scattered clouds,Clouds,0.000000,0.0
1,Jining,CN,2020-04-02 21:51:14,35.41,116.58,39.15,80,45,1.99,scattered clouds,Clouds,0.000000,0.0
2,Khatanga,RU,2020-04-02 21:51:14,71.97,102.50,15.87,91,100,18.07,overcast clouds,Clouds,0.000000,0.0
3,Belyy Yar,RU,2020-04-02 21:51:14,53.60,91.39,28.40,74,17,2.24,smoke,Smoke,0.000000,0.0
4,Vanimo,PG,2020-04-02 21:51:15,-2.67,141.30,79.32,84,99,8.77,light rain,Rain,0.062992,0.0


In [11]:
save_path = os.path.join('Resources','WeatherPy_challenge.csv')
city_data_df.to_csv(save_path)